# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [ ]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [2]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
train = pd.read_csv('q3_data/train.csv')
val = pd.read_csv('q3_data/val.csv')
test = pd.read_csv('q3_data/test.csv')

In [5]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [6]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [7]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [8]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [9]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [10]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [11]:
corpus = x_train + x_val + x_test

In [12]:
len(corpus)

12836

In [13]:
label_dict = {'False': 0, 'True': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'pants-fire': 5}
def encode(y):
  y_final = []
  for i in y:
    tmp = np.zeros(6)
    tmp[label_dict[i]] = 1
    y_final.append(tmp)
  return np.array(y_final)

In [14]:
y_train = encode(train['label'])
y_val = encode(val['label'])
y_test = encode(test['label'])

## Using bag-of-words embedding


In [15]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [16]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
X.shape

(12836, 8000)

In [18]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [19]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [20]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [21]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [ ]:
#!unzip "glove.6B.zip" -d "glove"

In [ ]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [ ]:
emmbed_dict['oov'] = np.zeros(200)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /home/avani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/home/avani/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,1), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,1), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,1), x_test2))

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [22]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [23]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [24]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dropout

model = keras.Sequential(
    [
     layers.Dense(units=4005,activation='relu'),
     layers.Dense(units=2000,activation='relu'),
     layers.Dense(units=1000,activation='relu'),
     layers.Dense(units=500, activation='relu'),
     layers.Dense(units=200, activation='relu'),
     layers.Dense(units=60, activation='relu'),
     layers.Dense(units=6, activation='softmax'),
     ]
)
model.compile(
    optimizer="rmsprop",
    loss="mse",
    metrics="accuracy"
)

In [25]:
#Bag of Words
h_bow = model.fit(
    x_train_bow, y_train_six_way,
    validation_data=(x_val_bow, y_val_six_way),
    batch_size=64,
    epochs=20,
)

Epoch 1/20
161/161 [==============================] - 11s 42ms/step - loss: 0.1316 - accuracy: 0.3158 - val_loss: 0.1208 - val_accuracy: 0.4213
Epoch 2/20
161/161 [==============================] - 6s 40ms/step - loss: 0.1076 - accuracy: 0.4654 - val_loss: 0.1168 - val_accuracy: 0.4260
Epoch 3/20
161/161 [==============================] - 6s 40ms/step - loss: 0.0906 - accuracy: 0.5468 - val_loss: 0.1209 - val_accuracy: 0.3995
Epoch 4/20
161/161 [==============================] - 6s 40ms/step - loss: 0.0746 - accuracy: 0.6320 - val_loss: 0.1364 - val_accuracy: 0.3980
Epoch 5/20
161/161 [==============================] - 7s 41ms/step - loss: 0.0643 - accuracy: 0.6753 - val_loss: 0.1372 - val_accuracy: 0.3964
Epoch 6/20
161/161 [==============================] - 6s 40ms/step - loss: 0.0574 - accuracy: 0.7152 - val_loss: 0.1732 - val_accuracy: 0.3879
Epoch 7/20
161/161 [==============================] - 7s 41ms/step - loss: 0.0540 - accuracy: 0.7305 - val_loss: 0.1350 - val_accuracy: 0.406

In [65]:
print("Accuracy of bag of words:", model.evaluate(x_test_bow, y_test_six_way, batch_size=64))

21/21 [==============================] - 0s 10ms/step - loss: 0.1683 - accuracy: 0.3640
Accuracy of bag of words: [0.1683412790298462, 0.3639906346797943]


In [ ]:
#Glove
h_glove = model.fit(
    x_train_glove, y_train_six_way,
    validation_data=(x_val_glove, y_val_six_way),
    batch_size=64,
    epochs=20,
)

#Bert Embedding
h_bert = model.fit(
    x_train_glove, y_train_six_way,
    validation_data=(x_val_glove, y_val_six_way),
    batch_size=64,
    epochs=20,
)

In [ ]:
print("Accuracy of Glove:", model.evaluate(x_test_glove, y_test_six_way, batch_size=64))
print()
print("Accuracy of bert:", model.evaluate(x_test_bert, y_test_six_way, batch_size=64))

In [37]:
from tensorflow.math import confusion_matrix
#Bag of words Confusion Matrix
bg_words = confusion_matrix(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_bow), axis=1))
print(bg_words)

<tf.Tensor: shape=(6, 6), dtype=int32, numpy=
array([[104,  37,  30,  39,  16,  24],
       [ 27,  64,  17,  56,  33,  14],
       [ 36,  34,  66,  47,  17,  14],
       [ 34,  59,  25, 120,  18,  11],
       [ 23,  55,  21,  73,  70,   7],
       [ 14,   5,  16,   9,   5,  43]], dtype=int32)>

In [ ]:
#Glove
glove = confusion_matrix(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_glove), axis=1))
#Bert
bert = confusion_matrix(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_bert), axis=1))

In [56]:
from sklearn.metrics import f1_score
#Bag of words
bg_f1 = f1_score(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_bow), axis=1), average=None)

In [ ]:
#Glove
glove_f1 = f1_score(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_glove), axis=1), average=None)
#Bert
bert_f1 = f1_score(np.argmax(y_test_six_way, axis=1), np.argmax(model.predict(x_test_bert), axis=1), average=None)

# Binary Classification

## Preprocessing

In [38]:
num_classes = 2

In [39]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [40]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [44]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dropout

model_binary = keras.Sequential(
    [
     layers.Dense(units=4005,activation='relu'),
     layers.Dense(units=2000,activation='relu'),
     layers.Dense(units=1000,activation='relu'),
     layers.Dense(units=500, activation='relu'),
     layers.Dense(units=200, activation='relu'),
     layers.Dense(units=60, activation='relu'),
     layers.Dense(units=2, activation='softmax'),
     ]
)
model_binary.compile(
    optimizer="rmsprop",
    loss="mse",
    metrics="accuracy"
)

In [45]:
#Bag of Words
print("\tBag of Words")
h_bow_binary = model_binary.fit(
    x_train_bow, y_train_binary,
    validation_data=(x_val_bow, y_val_binary),
    batch_size=64,
    epochs=20,
)
print()

	Bag of Words
Epoch 1/20
161/161 [==============================] - 8s 42ms/step - loss: 0.2201 - accuracy: 0.6747 - val_loss: 0.2026 - val_accuracy: 0.6449
Epoch 2/20
161/161 [==============================] - 7s 43ms/step - loss: 0.1660 - accuracy: 0.7557 - val_loss: 0.1945 - val_accuracy: 0.6986
Epoch 3/20
161/161 [==============================] - 7s 41ms/step - loss: 0.1333 - accuracy: 0.8035 - val_loss: 0.1985 - val_accuracy: 0.7157
Epoch 4/20
161/161 [==============================] - 7s 41ms/step - loss: 0.1021 - accuracy: 0.8513 - val_loss: 0.2272 - val_accuracy: 0.7040
Epoch 5/20
161/161 [==============================] - 6s 39ms/step - loss: 0.0862 - accuracy: 0.8779 - val_loss: 0.2210 - val_accuracy: 0.7150
Epoch 6/20
161/161 [==============================] - 6s 39ms/step - loss: 0.0751 - accuracy: 0.8970 - val_loss: 0.2428 - val_accuracy: 0.6838
Epoch 7/20
161/161 [==============================] - 6s 40ms/step - loss: 0.0666 - accuracy: 0.9069 - val_loss: 0.2534 - val_ac

In [68]:
print("Accuracy of bag of words in binary:", model_binary.evaluate(x_test_bow, y_test_binary, batch_size=64))

21/21 [==============================] - 0s 10ms/step - loss: 0.2691 - accuracy: 0.7054
Accuracy of bag of words in binary: [0.2691079080104828, 0.705377995967865]


In [63]:
#Bag of words Confusion Matrix
bg_words_binary = confusion_matrix(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_bow), axis=1))
print(bg_words_binary)

tf.Tensor(
[[384 172]
 [206 521]], shape=(2, 2), dtype=int32)


In [62]:
#Bag of Words F1 Score
bg_f1_binary = f1_score(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_bow), axis=1),  average=None)
print(bg_f1_binary)

[0.67015707 0.73380282]


In [ ]:
#Glove
print("\tGlove")
h_glove_binary = model.fit(
    x_train_glove, y_train_binary,
    validation_data=(x_val_glove, y_val_binary),
    batch_size=64,
    epochs=20,
)
print()
#Bert Embedding
print("\tBert Embedding")
h_bert_binary = model.fit(
    x_train_glove, y_train_binary,
    validation_data=(x_val_glove, y_val_binary),
    batch_size=64,
    epochs=20,
)

In [ ]:
#Glove accuracy
print("Accuracy of glove in binary:", model_binary.evaluate(x_test_glove, y_test_binary, batch_size=64))
#Bert accuracy
print("Accuracy of bert in binary:", model_binary.evaluate(x_test_bert, y_test_binary, batch_size=64))

In [ ]:
#Glove Confusion Matrix
glove_binary = confusion_matrix(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_glove), axis=1))
print(glove_binary)

#Bert Confusion Matrix
bert_binary = confusion_matrix(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_bert), axis=1))
print(bert_binary)

In [ ]:
#Bag of Words F1 Score
glove_f1_binary = f1_score(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_glove), axis=1),  average=None)
print(glove_f1_binary)

#Bag of Words F1 Score
bert_f1_binary = f1_score(np.argmax(y_test_binary, axis=1), np.argmax(model_binary.predict(x_test_bert), axis=1),  average=None)
print(bert_f1_binary)